# Analysis of Airbnb data for Amsterdam, Athens, Berlin
##### Data sourced from:
Gyódi, K., & Nawaro, Ł. (2021). Determinants of Airbnb prices in European cities: A spatial econometrics approach (Supplementary Material) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.4446043

In this report, we will use a real-world dataset, specfically on Airbnb listings in Amsterdam, Athens, and Berlin in 2019.

This dataset provides a comprehensive look at Airbnb prices in some of the most popular European cities. Each listing is evaluated for various attributes to capture an in-depth understanding of Airbnb prices on both weekdays and weekends. This data set can offer insight into how global markets are affected by social dynamics and geographical factors which in turn determine pricing strategies for optimal profitability.

In [ ]:
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(broom)
library(GGally)
library(AER)

In [ ]:
## Initial loading and wrangling. Ensure directory matches. 
amsterdam_weekdays <- read.csv("amsterdam_weekdays.csv") %>% as_tibble() %>% select(-X) %>% mutate(city = "amsterdam", day_type = "weekday")
amsterdam_weekends <- read.csv("amsterdam_weekends.csv") %>% as_tibble() %>% select(-X) %>% mutate(city = "amsterdam", day_type = "weekend")

athens_weekdays <- read.csv("athens_weekdays.csv") %>% as_tibble() %>% select(-X) %>% mutate(city = "athens", day_type = "weekday")
athens_weekends <- read.csv("athens_weekends.csv") %>% as_tibble() %>% select(-X) %>% mutate(city = "athens", day_type = "weekend")

berlin_weekdays <- read.csv("berlin_weekdays.csv") %>% as_tibble() %>% select(-X) %>% mutate(city = "berlin", day_type = "weekday")
berlin_weekends <- read.csv("berlin_weekends.csv") %>% as_tibble() %>% select(-X) %>% mutate(city = "berlin", day_type = "weekend")

airbnb <- bind_rows(amsterdam_weekdays, amsterdam_weekends, 
                   athens_weekdays, athens_weekends, 
                   berlin_weekdays, berlin_weekends) %>% 
                        mutate(room_type = as.factor(room_type), room_shared = as.factor(room_shared), 
                               multi = as.factor(multi), biz = as.factor(biz),
                               room_private = as.factor(room_private), host_is_superhost = as.factor(host_is_superhost), 
                                city = as.factor(city), day_type = as.factor(day_type))
head(airbnb)

In [ ]:
pair_plot <- airbnb %>% 
    select(person_capacity, cleanliness_rating, guest_satisfaction_overall, bedrooms) %>%
    ggpairs()

pair_plot

In [ ]:
# Main developer: Zhuo Liu

# Packages needed for LASSO

library(tidymodels)
library(glmnet)

In [ ]:
# Main developer: Zhuo Liu

# Use GVIF() to deselect variables whose GVIF is greater than sqrt(5)

set.seed(5033)

# Since room_shared and room_private are perfectly corrolated to room_type, we deselect room_shared and room_private.
airbnb_clean <- airbnb |>
    na.omit() |>
    select(-room_shared, -room_private)
    # mutate(room_sharedYes = if_else(room_shared == "True",1,0),
    #       room_privateYes = if_else(room_private == "True",1,0),
    #       host_is_superhostYes = if_else(host_is_superhost == "True",1,0),
    #       cityathens = if_else(city == "athens",1,0),
    #       cityberlin = if_else(city == "berlin",1,0),
    #       day_typeweekend = if_else(day_type == "weekend",1,0)) |>
    # select(-room_type, -room_shared, -room_private, -host_is_superhost, -city, -day_type)

head(airbnb_clean)

# Split data for variable selection to prevent double dipping
split <- initial_split(data = airbnb_clean, prop = 0.3)
variable_selection_df <- training(split)
inference_df <- testing(split)

# Calculate GVIF (right column) for each variable
vif_res <- glm(host_is_superhost ~ ., data = variable_selection_df, family = binomial) |>
    vif() |>
    round(4)

vif_res
# X_train <- as.matrix(training_df[,-1])
# Y_train <- as.matrix(training_df[,1])

# X_test <- as.matrix(testing_df[,-1])
# Y_test <- as.matrix(testing_df[,1])

# cv_LASSO <- cv.glmnet(
#   x = X_train, y = Y_train,
#   alpha = 1,
#   lambda = exp(seq(-21, 21, 0.1))
# )

# lambda <- cv_LASSO$lambda.min
# lambda

# coef(cv_LASSO, s = "lambda.min")

# test_pred_LASSO_min <- 
#             predict(cv_LASSO, 
#             newx = X_test, 
#             s = "lambda.min")
# LASSO_prediction <- tibble(Y_test,LASSO_prediction = test_pred_LASSO_min)
# head(LASSO_prediction)

In [ ]:
# Main developer: Zhuo Liu

# Use variables whose GVIF (right column) is less than or equal to sqrt(5) to conduct a hypothesis test

inference_model_for_superhost <- glm(host_is_superhost ~ .-attr_index
                                     -attr_index_norm
                                     -rest_index
                                     -rest_index_norm
                                     -lng 
                                     -lat
                                     -city, data = inference_df, family = binomial)

inference_model_for_superhost_res <- tidy(inference_model_for_superhost) |>
    mutate_if(is.numeric,round,4)

inference_model_for_superhost_res

vif_res_inference <- inference_model_for_superhost |>
    vif() |>
    round(4)

vif_res_inference






# Calculate test_RMSE of LASSO
# n <- nrow(LASSO_prediction)
# test_RMSE_LASSO <- LASSO_prediction |> mutate(Y_test = as.numeric(Y_test),
#                                        	LASSO_prediction = as.numeric(LASSO_prediction)) |>
#     summarize(RMSE = sqrt(sum((Y_test - LASSO_prediction) ^ 2) / n))
# test_RMSE_LASSO

In [ ]:
### Ridge
# cv_ridge <- cv.glmnet(
#   x = X_train, y = Y_train,
#   alpha = 0,
#   lambda = exp(seq(-5, 20, 0.1))
# )

# cv_lambda_ridge_min <- cv_ridge$lambda.min

# min_coef <- coef(cv_ridge, s = cv_lambda_ridge_min)

# full_OLS <- lm(realSum ~ ., data = training_df)

# ridge_coef <- cbind(
#   Full_OLS = coef(full_OLS),
#   Ridge_min = as.vector(min_coef)) %>%
#       round(4) %>% as.data.frame()

# test_pred_ridge_min <- 
#             predict(cv_ridge, 
#             newx = X_test, 
#             s = cv_lambda_ridge_min)

# ridge_prediction_table <-tibble(Y_test,ridge_prediction = test_pred_ridge_min)

# ridge_prediction_table %>% head()


In [ ]:
### Ride RMSE

# test_RMSE_ridge <- ridge_prediction_table %>% mutate(Y_test = as.numeric(Y_test),
#                                        	ridge_prediction = as.numeric(ridge_prediction)) %>%
#     summarize(RMSE = sqrt(sum((Y_test - ridge_prediction) ^ 2) / nrow(ridge_prediction_table)))
# test_RMSE_ridge
